In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [8]:
import http.client
import json
import pandas as pd
import csv
import psycopg2

In [50]:
import requests

url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

querystring = {"limit":"1000"}

headers = {
	"x-rapidapi-key": "bc68487461mshca1d2214a6e1c46p1c3c47jsn5f4df3a84144",
	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())


data= response.json()

#Saving into a file
filename = "PropertyRecords.json"
with open(filename,"w") as file:
    json.dump(data,file,indent=4)


[{'bathrooms': 3, 'bedrooms': 4, 'squareFootage': 4010, 'county': 'Monmouth', 'propertyType': 'Land', 'addressLine1': '104 Peskin Rd', 'city': 'Farmingdale', 'state': 'NJ', 'zipCode': '07727', 'formattedAddress': '104 Peskin Rd, Farmingdale, NJ 07727', 'lotSize': 204732, 'yearBuilt': 1940, 'features': {'architectureType': 'Colonial', 'cooling': True, 'coolingType': 'Commercial', 'exteriorType': 'Aluminum / Vinyl Siding', 'fireplace': True, 'floorCount': 2, 'foundationType': 'Block', 'garage': True, 'garageType': 'Detached', 'heating': True, 'heatingType': 'Forced Air', 'roofType': 'Shingle', 'roomCount': 10, 'unitCount': 1}, 'assessorID': '21  00154-0000-00006', 'legalDescription': 'LOT:6 BLK:154 DIST:21 CITY/MUNI/TWP:HOWELL TWP MAP REF:TAX MAP 6.17', 'subdivision': 'TOWNSHIP/HOWELL', 'zoning': 'ARE2', 'taxAssessment': {'2023': {'value': 646200, 'land': 206600, 'improvements': 439600}}, 'propertyTaxes': {'2022': {'total': 11781}}, 'lastSalePrice': 750000, 'lastSaleDate': '2023-05-26T00

In [51]:
#Read into a dataframe

Propertyrecord_df = pd.read_json('PropertyRecords.json')

In [53]:
Propertyrecord_df.info()
print(Propertyrecord_df.columns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bathrooms         8 non-null      float64
 1   bedrooms          8 non-null      float64
 2   squareFootage     8 non-null      float64
 3   county            10 non-null     object 
 4   propertyType      8 non-null      object 
 5   addressLine1      10 non-null     object 
 6   city              10 non-null     object 
 7   state             10 non-null     object 
 8   zipCode           10 non-null     int64  
 9   formattedAddress  10 non-null     object 
 10  lotSize           8 non-null      float64
 11  yearBuilt         8 non-null      float64
 12  features          10 non-null     object 
 13  assessorID        8 non-null      object 
 14  legalDescription  8 non-null      object 
 15  subdivision       8 non-null      object 
 16  zoning            5 non-null      object 
 17  

In [57]:
#Transformational 

#Convert columns with dictionary into string
import json

Propertyrecord_df['features']=Propertyrecord_df ['features'].apply(json.dumps)

#fill missing nd NA values
Propertyrecord_df.fillna(
    {
        'bathrooms': 0,
        'bedrooms': 0,
        'squareFootage': 0, 
        'county': "Not Available", 
        'propertyType': "unknown",
        'addressLine1': "unknown",
        'city': "unknown", 
        'state': "unknown", 
        'zipCode': "unknown", 
        'formattedAddress': "Not Available",
        'yearBuilt': 0, 
        'features': "unknown", 
        'assessorID': "unknown", 
        'legalDescription': "Not Available",
        'subdivision': "Not Available", 
        'ownerOccupied': 0, 
        'lotSize': 0, 
        'taxAssessment': "Not Available",
        'propertyTaxes': "Not Available", 
        'lastSaleDate': "Not Available", 
        'lastSalePrice': 0, 
        'owner': "unknown", 
        'id': "unknown",
        'longitude': "unknown", 
        'latitude': "unknown", 
        'zoning': "unknown", 
        'addressLine2': "Not Available"
    },
    inplace=True
)


In [59]:
print(Propertyrecord_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bathrooms         10 non-null     float64
 1   bedrooms          10 non-null     float64
 2   squareFootage     10 non-null     float64
 3   county            10 non-null     object 
 4   propertyType      10 non-null     object 
 5   addressLine1      10 non-null     object 
 6   city              10 non-null     object 
 7   state             10 non-null     object 
 8   zipCode           10 non-null     int64  
 9   formattedAddress  10 non-null     object 
 10  lotSize           10 non-null     float64
 11  yearBuilt         10 non-null     float64
 12  features          10 non-null     object 
 13  assessorID        10 non-null     object 
 14  legalDescription  10 non-null     object 
 15  subdivision       10 non-null     object 
 16  zoning            10 non-null     object 
 17  

In [61]:
#create the FACT Table
fact_columns =['addressLine1','city', 'state','zipCode', 'formattedAddress',  'squareFootage', 'yearBuilt', 'bathrooms', 'bedrooms', 'lotSize', 'propertyType',
                'longitude', 'latitude']

fact_table = Propertyrecord_df[fact_columns]

#Create Location Dimension
Location_dim= Propertyrecord_df[['addressLine1','city', 'state','zipCode','county','longitude', 'latitude']].drop_duplicates().reset_index(drop=True)
Location_dim.index.name= 'location_id'

In [63]:
#Create Sales Dimension

sales_dim = Propertyrecord_df[['lastSalePrice','lastSaleDate']].drop_duplicates().reset_index(drop=True)
sales_dim.index.name= 'sales_id'
sales_dim.head()

,lastSalePrice,lastSaleDate
sales_id,,
0,750000.0,2023-05-26T00:00:00.000Z
1,172000.0,2019-10-09T00:00:00.000Z
2,0.0,2019-12-12T00:00:00.000Z
3,0.0,Not Available
4,105000.0,1999-04-09T00:00:00.000Z


In [65]:
#Create Property Features Dimension
features_dim = Propertyrecord_df[['features', 'propertyType', 'zoning']].drop_duplicates().reset_index(drop=True)
features_dim.index.name ='features_id'
features_dim.head()


,features,propertyType,zoning
features_id,,,
0,"{""architectureType"": ""Colonial"", ""cooling"": tr...",Land,ARE2
1,"{""architectureType"": ""Split Foyer"", ""exteriorT...",Single Family,R-1
2,"{""architectureType"": ""Ranch"", ""fireplace"": tru...",Single Family,INC
3,{},unknown,unknown
4,"{""floorCount"": 2, ""roofType"": ""Shingle"", ""unit...",Single Family,unknown


In [29]:

#Output to csv
fact_table.to_csv('property_fact.csv', index=False)
Location_dim.to_csv('Location_dimension.csv', index=True)
sales_dim.to_csv('sales_dimension.csv', index=True)
features_dim.to_csv('features_dimension.csv', index=True)

In [67]:
  #Loading Layer

# develop a function to connect to pgadmin

def get_db_connection():
    connection = psycopg2.connect(
        host= 'localhost',
        database='postgres',
        user='postgres',
        password='shakirat12'
    )
    return connection

conn = get_db_connection()


In [87]:
#creating tables


def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    create_table_query='''CREATE SCHEMA IF NOT EXISTS zapbank;

                            DROP TABLE IF EXISTS zapbank.fact_table;
                            DROP TABLE IF EXISTS zapbank.location_dim;
                            DROP TABLE IF EXISTS zapbank.sales_dim;
                            DROP TABLE IF EXISTS zapbank.features_dim;

                            CREATE TABLE zapbank.fact_table(
                                addressline1 VARCHAR(255),
                                city VARCHAR(100),
                                state VARCHAR(50),
                                zipCode INTEGER,
                                formattedAddress VARCHAR(255),
                                squareFootage FLOAT,
                                yearBuilt FLOAT,
                                bathrooms FLOAT,
                                bedrooms FLOAT,
                                lotSize FLOAT,
                                propertyType VARCHAR(255),
                                longitude FLOAT,
                                latitude FLOAT
                                );

                                CREATE TABLE zapbank.location_dim(
                                    location_id SERIAL PRIMARY KEY,
                                    addressLine1 VARCHAR(255),
                                    city VARCHAR(100),
                                    state VARCHAR (50),
                                    zipCode INTEGER,
                                    county VARCHAR(100),
                                    longitude FLOAT,
                                    latitude FLOAT
                                    );

                                CREATE TABLE zapbank.sales_dim(
                                   sales_id SERIAL PRIMARY KEY,
                                   lastSalePrice FLOAT,
                                   lastSaleDate DATE
                                   );

                                CREATE TABLE zapbank.features_dim(
                                    features_id SERIAL PRIMARY KEY,
                                    features TEXT,
                                    propertyType VARCHAR(255),
                                    zoning VARCHAR(255)
                                     );'''
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()
create_tables()

 


In [89]:
#Create a funcion to load csv data from a folder into the DB
def load_data_from_csv_to_table(csv_path,table_name):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(csv_path,'r',encoding='utf-8') as file:
        reader=csv.reader(file)
        next(reader) #Skip the header row
        for row in reader:
            placeholders = ', '.join(['%s']* len(row))
            query = f"INSERT INTO {table_name} VALUES ({placeholders});"
            cursor.execute(query,row)
        conn.commit()
        cursor.close()
        conn.close()



In [91]:
# for fact table
fact_csv_path = r'C:\Users\HP\Documents\Data Engineering Projects\property_fact.csv'
load_data_from_csv_to_table(fact_csv_path, 'zapbank.fact_table')

In [93]:
# for Location dimension table
location_csv_path = r'C:\Users\HP\Documents\Data Engineering Projects\Location_dimension.csv'
load_data_from_csv_to_table(location_csv_path, 'zapbank.location_dim')

In [97]:
# for features dimension table
features_csv_path = r'C:\Users\HP\Documents\Data Engineering Projects\features_dimension.csv'
load_data_from_csv_to_table(features_csv_path, 'zapbank.features_dim')

In [121]:
 #Create a New funcion to load csv data for sales  from a folder into the DB
def load_data_from_csv_to_sales_table(csv_path,table_name):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(csv_path,'r',encoding='utf-8') as file:
        reader=csv.reader(file)
        next(reader) #Skip the header row
        for row in reader:
            #convert empty strings (or Not available) in date column to None (NULL in sql)
            row=[None if (cell == '' or cell =='Not Available') and col_name == 'lastSaleDate' else cell for cell, col_name in zip(row, sales_dim_columns)]
            placeholders = ', '.join(['%s']* len(row))
            query = f"INSERT INTO {table_name} VALUES ({placeholders});"
            cursor.execute(query,row)
        conn.commit()
        cursor.close()
        conn.close()

#define the columns names in sales_dim table
sales_dim_columns = ['sales_id','lastSalePrice', 'lastSaleDate']



In [125]:
# for sales dimension table
sales_csv_path = r'C:\Users\HP\Documents\Data Engineering Projects\sales_dimension.csv'
load_data_from_csv_to_sales_table(sales_csv_path, 'zapbank.sales_dim')

UniqueViolation: duplicate key value violates unique constraint "sales_dim_pkey"
DETAIL:  Key (sales_id)=(0) already exists.


In [127]:
print('All data has been loaded successfully into their respective schema and table')

All data has been loaded successfully into their respective schema and table
